In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, models
from torchvision import transforms
from torch.optim import lr_scheduler


In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
device

In [ ]:
transform = transforms.Compose([
    transforms.Resize(224)
    transforms.ToTensor(),
    transforms.Normalize(
      mean = [0.5071, 0.4867, 0.4408],
      std = [0.2675, 0.2565, 0.2761])
    ])

train_dataset = torchvision.datasets.CIFAR100(transform = transform, download=True, root = "data", train = True)
test_dataset = torchvision.datasets.CIFAR100(transform = transform, download=True, root = "data", train = False)
len(train_dataset.classes)



In [ ]:
batch_size = 64

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

model = models.resnet18(pretrained = True)
num_features = model.fc.in_features
model.fc = nn.Linear(in_features = num_features, out_features = 100)

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

for param in model.layer4.parameters()
    param.requires_grad = True

for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

In [ ]:
from torch.optim import lr_scheduler


model = model.to(device)

criterion = nn.CrossEntropyLoss()
optim = optim.SGD(model.fc.parameters(), lr = 0.01, momentum = 0.9)
scheduler = lr_scheduler.StepLR(optim, 7, 0.1)


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optim.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optim.step()

        running_loss += loss.item()

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
      for inputs, labels in test_loader:
        outputs = model(inputs.to(device))
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy}%')
    scheduler.step(val_loss)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {running_loss / len(train_loader)}, Val Loss: {val_loss}")
